# RADIA Python Example 2: ESRF Superconducting Wiggler

This example creates a stack of four racetrack coils, one circular coil,
and reflects them in the (_x_,_y_) plane.
This geometry corresponds to a 4&#160;T superconducting wiggler that used
to be in operation at the ESRF.

In [ ]:
%matplotlib inline
from __future__ import print_function #Py 2.*/3.* compatibility
import radia as rad
import numpy as np
import matplotlib.pyplot as plt
#import uti_plot

# until we have 3D graphics, simply
# import an illustration of this magnet
from IPython.display import Image
Image(filename=('./RadiaSCW.png'))

We first define a function that constructs the desired magnet geometry in Radia.

In [ ]:
# define function to build this magnet
def buildMagnetSCW():
    # current densities in A / mm^2
    j1 = 128; j2 = 256
    
    # number of arc segments
    n1 = 3; n2 = 6

    # create 5 racetrack coils above the mid-plane:
    #   lower inside, lower outside, upper inside, upper outside, and circular
    # rad.ObjRaceTrk[ctr:[x,y,z], rad:[r1,r2], lstr:[lx,ly], ht, nseg, j]
    rt1 = rad.ObjRaceTrk([0.,0.,38.], [  9.5, 24.5], [120.,0.], 36, n1,  j1)
    rt2 = rad.ObjRaceTrk([0.,0.,38.], [ 24.5, 55.5], [120.,0.], 36, n1,  j2)
    rt3 = rad.ObjRaceTrk([0.,0.,76.], [ 10.0, 25.0], [ 90.,0.], 24, n1,  j1)
    rt4 = rad.ObjRaceTrk([0.,0.,76.], [ 25.0, 55.0], [ 90.,0.], 24, n1,  j2)
    rt5 = rad.ObjRaceTrk([0.,0.,60.], [150.0,166.3], [  0.,0.], 39, n2, -j2)

    # set colors and line thickness (must we do this here?!?)
    #c1 = [0.0,1.0,1.0] # blue/green
    #c2 = [1.0,0.4,0.0] # orange-red
    #thcn = 0.001
    #rad.ObjDrwAtr(rt1, c1, thcn)
    #rad.ObjDrwAtr(rt2, c2, thcn)
    #rad.ObjDrwAtr(rt3, c1, thcn)
    #rad.ObjDrwAtr(rt4, c2, thcn)
    #rad.ObjDrwAtr(rt5, c2, thcn)

    # assemble into a group
    grp = rad.ObjCnt([rt1, rt2, rt3, rt4, rt5])

    # and reflect in the (x,y) plane [plane through (0,0,0) with normal (0,0,1)]
    rad.TrfZerPara(grp, [0,0,0], [0,0,1])

    return grp

We next define a function that uses Radia to compute the field and field integral values of interest.

In [ ]:
# define function to compute the desired values
# of magnetic field and magnetic field integral
def calcField(g):

    # vertical field as a function of Y
    yMin = -300.; yMax = 300.; ny = 301
    yStep = (yMax - yMin)/(ny - 1)
    xc = 0.; zc = 0.
    BzVsY = rad.Fld(g, 'bz', [[xc, yMin + iy * yStep, zc] for iy in range(ny)])

    # mid-plane vertical field integrated along Y as a function of X
    xMin = -400.; xMax = 400.; nx = 201
    xStep = (xMax - xMin)/(nx - 1)
    zc = 0.
    IBzVsX = [rad.FldInt(g, 'inf', 'ibz', [xMin + ix * xStep, -300., zc], [xMin + ix * xStep, 300., zc])
                        for ix in range(nx)]
    
    return BzVsY, [yMin, yMax, ny], IBzVsX, [xMin, xMax, nx]

Now exercise the above functions:

In [ ]:
# build the magnet
g = buildMagnetSCW()
#print('SCW Geometry Index:', g)

## display 3D magnet geometry
## set colors and line thickness
#c1 = [0.0,1.0,1.0] # blue/green
#c2 = [1.0,0.4,0.0] # orange-red
#thcn = 0.001
#rad.ObjDrwAtr(rt1, c1, thcn)
#rad.ObjDrwAtr(rt2, c2, thcn)
#rad.ObjDrwAtr(rt3, c1, thcn)
#rad.ObjDrwAtr(rt4, c2, thcn)
#rad.ObjDrwAtr(rt5, c2, thcn)
#rad.ObjDrwOpenGL(g)

In [ ]:
# calculate values of magnetic field and field integral
BzVsY, MeshY, IBzVsX, MeshX = calcField(g)

# print central values
print('    Field Value at Center:', BzVsY[0],  'T')
print(' Field Integral at Center:', IBzVsX[0], 'T.mm')

# and plot the results
plt.figure(figsize=(7,4.3))
plt.plot(np.linspace(MeshY[0],MeshY[1],MeshY[2]), BzVsY)
plt.title(r'Vertical Field $B_z(y)$')
plt.xlabel(r'longitudinal position, $y$ / mm')
plt.ylabel(r'$B_z(y)$ / T')
plt.grid()
plt.show()

plt.figure(figsize=(7,4.3))
plt.plot(np.linspace(MeshX[0],MeshX[1],MeshX[2]), IBzVsX)
plt.title(r'Vertical Magnetic Field Integral, $\int B_z(y)\,\mathrm{d}y$')
plt.xlabel(r'horizontal position, $x$ / mm')
plt.ylabel(r'$\int B_z(y)\,\mathrm{d}y$ / T$\cdot$mm')
plt.grid()
plt.show()
#plt.xlim([0,xmax])
#plt.ylim([0,ymax])
##plt.legend()
##plt.tight_layout()
##plt.savefig('foo.png')